<a href="https://colab.research.google.com/github/shreyakurani/Shreya_HDS_5230_07/blob/main/week_05_ipynb_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Obtaining the dataset

In [1]:
!wget "https://coronadatascraper.com/timeseries.csv.zip"

--2025-02-23 18:39:15--  https://coronadatascraper.com/timeseries.csv.zip
Resolving coronadatascraper.com (coronadatascraper.com)... 104.21.48.1, 104.21.112.1, 104.21.32.1, ...
Connecting to coronadatascraper.com (coronadatascraper.com)|104.21.48.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6569453 (6.3M) [application/zip]
Saving to: ‘timeseries.csv.zip’

timeseries.csv.zip  100%[===================>]   6.26M  19.8MB/s    in 0.3s    

2025-02-23 18:39:16 (19.8 MB/s) - ‘timeseries.csv.zip’ saved [6569453/6569453]



# 2. Extracting the .csv file and loading it into a Dask dataframe

In [2]:
!unzip timeseries.csv.zip

Archive:  timeseries.csv.zip
  inflating: timeseries.csv          


In [3]:
import dask.dataframe as dd
df = dd.read_csv('timeseries.csv') # This was consistently giving error so I changed the approach by manually specifying dtypes
dtypes = {
    'cases': 'float64',
    'county': 'object',
    'deaths': 'float64',
    'recovered': 'float64',
    'state': 'object'
}
df = dd.read_csv('timeseries.csv', dtype=dtypes)
print(df.head())

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


  locationID slug     name    level  city county state  country    lat  long  \
0    iso1:ad   ad  Andorra  country   NaN   <NA>  <NA>  Andorra  42.55  1.58   
1    iso1:ad   ad  Andorra  country   NaN   <NA>  <NA>  Andorra  42.55  1.58   
2    iso1:ad   ad  Andorra  country   NaN   <NA>  <NA>  Andorra  42.55  1.58   
3    iso1:ad   ad  Andorra  country   NaN   <NA>  <NA>  Andorra  42.55  1.58   
4    iso1:ad   ad  Andorra  country   NaN   <NA>  <NA>  Andorra  42.55  1.58   

   ...  deaths  recovered active  tested  hospitalized  hospitalized_current  \
0  ...     0.0        0.0    NaN     NaN           NaN                   NaN   
1  ...     0.0        0.0    NaN     NaN           NaN                   NaN   
2  ...     0.0        0.0    NaN     NaN           NaN                   NaN   
3  ...     0.0        0.0    NaN     NaN           NaN                   NaN   
4  ...     0.0        0.0    NaN     NaN           NaN                   NaN   

   discharged  icu  icu_current       

# 3.1 Creating  new dataframe object with samples that correspond to states in US.

In [4]:
us_states_df = df[(df['country'] == 'United States') & (df['level'] == 'state')]
print(us_states_df.head())
print(us_states_df.columns)

                locationID       slug                   name  level  city  \
240069  iso1:us#iso2:us-ak  alaska-us  Alaska, United States  state   NaN   
240070  iso1:us#iso2:us-ak  alaska-us  Alaska, United States  state   NaN   
240071  iso1:us#iso2:us-ak  alaska-us  Alaska, United States  state   NaN   
240072  iso1:us#iso2:us-ak  alaska-us  Alaska, United States  state   NaN   
240073  iso1:us#iso2:us-ak  alaska-us  Alaska, United States  state   NaN   

       county   state        country    lat    long  ...  deaths  recovered  \
240069   <NA>  Alaska  United States  64.24 -152.23  ...     NaN        NaN   
240070   <NA>  Alaska  United States  64.24 -152.23  ...     NaN        NaN   
240071   <NA>  Alaska  United States  64.24 -152.23  ...     NaN        NaN   
240072   <NA>  Alaska  United States  64.24 -152.23  ...     NaN        NaN   
240073   <NA>  Alaska  United States  64.24 -152.23  ...     NaN        NaN   

       active  tested  hospitalized  hospitalized_current  dis

In [5]:
print(us_states_df['state'].compute().unique())

<ArrowStringArray>
[                      'Alaska',                      'Alabama',
                     'Arkansas',               'American Samoa',
                      'Arizona',                   'California',
                     'Colorado',                  'Connecticut',
             'Washington, D.C.',                     'Delaware',
                      'Florida',                      'Georgia',
                         'Guam',                       'Hawaii',
                         'Iowa',                        'Idaho',
                     'Illinois',                      'Indiana',
                       'Kansas',                     'Kentucky',
                    'Louisiana',                'Massachusetts',
                     'Maryland',                        'Maine',
                     'Michigan',                    'Minnesota',
                     'Missouri',     'Northern Mariana Islands',
                  'Mississippi',                      'Montana',
      

There are only 50 US States, here we have federal district (Washington, D.C.), and other US territories (American Samoa, Guam, Northern Mariana Islands, Puerto Rico, and the United States Virgin Islands). Which makes our state list 56.

In [6]:
states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
    'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
    'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
    'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
    'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
    'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
    'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
    'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
    'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
]

us_states_df = df[(df['country'] == 'United States') &
                  (df['level'] == 'state') &
                  (df['state'].isin(states))]
print(us_states_df['state'].compute().unique())
print(us_states_df.head())

<ArrowStringArray>
[        'Alaska',        'Alabama',       'Arkansas',        'Arizona',
     'California',       'Colorado',    'Connecticut',       'Delaware',
        'Florida',        'Georgia',         'Hawaii',           'Iowa',
          'Idaho',       'Illinois',        'Indiana',         'Kansas',
       'Kentucky',      'Louisiana',  'Massachusetts',       'Maryland',
          'Maine',       'Michigan',      'Minnesota',       'Missouri',
    'Mississippi',        'Montana', 'North Carolina',   'North Dakota',
       'Nebraska',  'New Hampshire',     'New Jersey',     'New Mexico',
         'Nevada',       'New York',           'Ohio',       'Oklahoma',
         'Oregon',   'Pennsylvania',   'Rhode Island', 'South Carolina',
   'South Dakota',      'Tennessee',          'Texas',           'Utah',
       'Virginia',        'Vermont',     'Washington',      'Wisconsin',
  'West Virginia',        'Wyoming']
Length: 50, dtype: string
                locationID       slug     

# 3.2 Computing per-capita mortality

In [7]:
us_states_df['date'] = dd.to_datetime(us_states_df['date'])

filtered_df = us_states_df[(us_states_df['date'] >= '2020-01-01') & (us_states_df['date'] <= '2021-02-28')]

agg_df = filtered_df.groupby('state').agg({'deaths': 'sum', 'population': 'mean'}).reset_index()

agg_df['per_capita_mortality'] = agg_df['deaths'] / agg_df['population']

result_df = agg_df.sort_values(by='per_capita_mortality', ascending=False)

print(result_df.compute())


             state     deaths  population  per_capita_mortality
30      New Jersey  2418962.0   8882190.0              0.272338
33        New York  4402011.0  19453561.0              0.226283
6      Connecticut   691521.0   3565287.0              0.193959
18   Massachusetts  1316575.0   6892503.0              0.191016
38    Rhode Island   144884.0   1059361.0              0.136765
17       Louisiana   614872.0   4648794.0              0.132265
21        Michigan  1070947.0   9986857.0              0.107236
13        Illinois  1121626.0  12671821.0              0.088513
19        Maryland   517991.0   6045680.0              0.085680
24     Mississippi   251974.0   2976149.0              0.084664
37    Pennsylvania  1068206.0  12801989.0              0.083441
7         Delaware    74695.0    973764.0              0.076707
3          Arizona   469482.0   7278717.0              0.064501
9          Georgia   621536.0  10617423.0              0.058539
8          Florida  1084199.0  21477737.

# 3.3 Calculating CFR using WHO Scientific Brief

## Assumptions Made
- Cumulative Reporting: We assume that the daily reported 'cases' and 'deaths' columns are cumulative counts.
- End-of-Month Totals: We assume that taking the maximum value within a month accurately represents the cumulative totals at the end of that month.
- Resolved Cases: We assume that by the end of each month, the majority of cases are resolved, allowing the ratio of deaths to cases to serve as an approximation for the CFR.
- Consistent Detection: We assume that the detection and reporting practices are sufficiently consistent over the time period, despite known biases early in the outbreak.

In [8]:
filtered_df['year_month'] = filtered_df['date'].dt.to_period('M')

monthly_agg = filtered_df.groupby(['state', 'year_month']).agg({'cases': 'max', 'deaths': 'max'}).reset_index()

monthly_agg['CFR'] = monthly_agg['deaths'] / monthly_agg['cases']

monthly_pd = monthly_agg.compute()

cfr_pivot = monthly_pd.pivot(index='state', columns='year_month', values='CFR')

cfr_pivot


year_month,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10
state,,,,,,,,,,
Alabama,NaN,NaN,0.013252,0.038325,0.035245,0.024970,0.018011,0.017309,0.016419,0.016022
Alaska,NaN,NaN,0.025210,0.025352,0.023041,0.014894,0.006255,0.006041,0.007157,0.006734
Arizona,NaN,NaN,0.018619,0.041841,0.045445,0.020602,0.021229,0.024921,0.025857,0.025815
Arkansas,NaN,NaN,0.015296,0.018740,0.018337,0.012995,0.010656,0.013018,0.016357,0.016550
California,NaN,NaN,0.021279,0.040429,0.037282,0.026362,0.018412,0.018280,0.019393,0.019355
Colorado,NaN,NaN,0.019414,0.051904,0.045253,0.046384,0.039281,0.033883,0.029086,0.028144
Connecticut,NaN,NaN,0.022059,0.081480,0.093458,0.092918,0.088978,0.084438,0.081990,0.079869
Delaware,NaN,NaN,0.037618,0.044360,0.049274,0.044361,0.039559,0.034712,0.033264,0.034012
Florida,NaN,NaN,0.012149,0.038290,0.045119,0.023643,0.014548,0.017943,0.020264,0.020368


As you can see that we do not have any data after 2020-10. Moving ahead with that.

# 3.4 Using the matrix as input, computing the states ranking on how CFR rate changed over time
## Assumptions Made
- CFR Calculation: We assume that the provided CFR matrix was computed by taking the cumulative deaths divided by cumulative cases (using end-of-month values).
- Month-to-Month Change: The change in CFR for a month is defined as the difference between that month’s CFR and the previous month’s CFR.
- Aggregation:
    - When summing the raw differences (net change), the overall value reflects the change from the first to the last month.
    - When summing the absolute differences (total change), the value captures the total magnitude of changes (volatility) without canceling out increases and decreases.
- Ranking Purpose: Depending on whether you care about net trends or overall volatility, you can choose either ranking metric.

In [9]:
monthly_diff = cfr_pivot.diff(axis=1)

net_change = monthly_diff.sum(axis=1)

total_change = monthly_diff.abs().sum(axis=1)

ranking_net = net_change.sort_values(ascending=False)

ranking_total = total_change.sort_values(ascending=False)

print("Ranking by net change (final minus initial CFR):")
print(ranking_net)

print("\n\n\n\n")

print("Ranking by total change (sum of absolute month-to-month changes):")
print(ranking_total)


Ranking by net change (final minus initial CFR):
state
Connecticut       0.057810
New York          0.050899
Massachusetts     0.050333
New Jersey        0.049363
New Hampshire     0.041598
Pennsylvania      0.035525
Indiana           0.030448
Rhode Island      0.030016
Michigan          0.020535
West Virginia     0.015320
New Mexico        0.014950
Illinois          0.012510
North Carolina    0.011212
Maine             0.009015
Colorado          0.008730
Mississippi       0.008569
Florida           0.008219
Texas             0.008140
Wyoming           0.007995
Minnesota         0.007281
Hawaii            0.007267
Arizona           0.007196
Ohio              0.005814
Missouri          0.005613
Maryland          0.005092
Alabama           0.002771
Tennessee         0.002706
South Carolina    0.002399
Arkansas          0.001254
Iowa              0.000890
South Dakota      0.000823
Utah              0.000486
Wisconsin         0.000184
Virginia         -0.000116
California       -0.001924


# Reasoning

### 1. Filtering and Grouping on the Full Timeseries Dataset
Working with a big COVID-19 dataset, which could have millions of rows, needs smart processing to make things faster and easier. A key step is changing the date column into a proper date format, picking only the dates needed, and then sorting the data by state and month to find total counts. Since the dataset is so large, using Dask is very helpful because it allows the computer to process parts of the data at the same time, using multiple cores or even several computers. This helps save memory and speeds up reading and writing data by handling smaller chunks at once. Even though working in parallel has some extra setup, the boost in speed and efficiency makes it a great choice for dealing with big amounts of data.


### 2. Aggregating to Compute CFR per Month
After organizing the COVID-19 data by finding the highest number of cases and deaths in each state per month, the next step is to reshape the data and figure out the case fatality rate (CFR). This process makes the dataset much smaller, usually into a simple table with about 50 rows and 10 columns. At first, using multiple computer cores helps speed up sorting the full dataset, but once it's smaller, a regular computer can handle it easily. So, switching to a Pandas DataFrame is a good idea because it removes extra computing steps and makes the data easier to work with quickly.


### 3. Computing Month-to-Month Differences and Aggregating Changes
To understand how COVID-19 cases changed in different states, a helpful step is to find the difference in cases from one month to the next using the `diff()` function. Then, we add up both the total changes and the absolute changes, and rank states based on these numbers. Since the dataset is small, with only 50 states over 14 months, these calculations are very quick. Simple tasks like finding differences, adding values, and sorting can be done instantly using Pandas. Because of this, using multiple computers or parallel processing isn’t needed—it would actually take longer to set up than just running everything on a regular computer.


### Overall Reasoning
When working with COVID-19 data, the way we process it depends on how big the dataset is. In the beginning, when we need to sort, change, and group large amounts of data, using a tool like Dask is helpful because it can split the work across multiple computers. This makes handling a huge dataset much easier. But once we shrink the data down to a smaller summary table, like a 50 × 10 chart for case fatality rates (CFR), we don’t need that extra power anymore. At this stage, using Pandas on a single computer is much faster and simpler because setting up a big system would just slow things down without giving any real benefits.
